To Do:
- Have scaper identify and download forms in formats other than pdfs (e.g., doc and docx)
- Add more jurisdictions/states

# Load Libraries

In [1]:
import urllib.request, json 
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import date
import os
from os import walk
import os.path
from os import path

In [2]:
def reCase(text):
    output = re.sub("(\w|\d)(_|-)(\w|\d)","\\1 \\3",text)
    output = re.sub("([a-z])([A-Z]|\d)","\\1 \\2",output)
    output = re.sub("(\d)([A-Z]|[a-z])","\\1 \\2",output)
    return output

In [18]:
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text

#def download_pdf(url,path,filename):
#    filename = path+filename
#    try:
#        f = urlopen(url)
#        with open(filename, "wb") as code:
#            code.write(f.read())
#        return filename
#    except:
#        return "error"
    
def download_pdf(url,path,filename):
    try:
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

# Download forms and meta data from UT courts

Download forms and place them in the folder defined by `form_path` while creating a dataframe that you save to `csv_path`.


In [41]:
source = "www.utcourts.gov"
site = "https://www.utcourts.gov/forms/"
form_path = "../data/raw/www.utcourts.gov/forms/"
csv_path = "../data/raw/www.utcourts.gov/"

try:
    files_df = pd.read_csv(csv_path+"form_data.csv")
except:
    files_df = pd.DataFrame([],columns=["source","title","group","url","filename","downloaded"])
    files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")    

In [5]:
with urllib.request.urlopen(urllib.request.Request(site)) as url:
    content = url.read().decode()
soup = BeautifulSoup(content)

In [19]:
today = date.today().strftime("%Y-%m-%d")
body = soup.find('div',id="content")
for section in re.split("<h3>",str(body), flags=re.IGNORECASE):
    group = re.match('^[^<]*',section)[0]
    
    #print(group)
    
    section_html = BeautifulSoup(section)
    for li in section_html.find_all('li'):
        title = re.sub('\(.*\s*.*','',li.text)
        
        #print("\t",i,title)
        
        for a in li.find_all('a', href=True):
            if (re.search("^http.*\.pdf",a['href'])):
                url = re.search(".*/([^/]*)\.pdf",a['href'], re.IGNORECASE)
                filename = url.group(1)+".pdf"
                fileurl = url.group(0)
                
                #print("\t",filename,"\n\t",fileurl,"\n")
                
                status = ""
                if fileurl not in files_df["url"].values:
                    print("Downloading: "+str(hash(url))+" "+fileurl)
                    status = download_pdf(fileurl,form_path,str(hash(url))+".pdf")

                    if status == "error":
                        print("#######################")
                        print("         Error!!!")
                        print("#######################")
                        #time.sleep(5)
                    else:
                        files_df = files_df.append(pd.DataFrame([[source,title,group,fileurl,filename,today]],columns=["source","title","group","url","filename","downloaded"]), ignore_index=True,sort=False)
                        #time.sleep(1)

Downloading: 8791454367091 https://www.utcourts.gov/mediation/cpm/docs/CPMMotiontoEnforceParenttime.pdf
#######################
         Error!!!
#######################
Downloading: 8791454474435 https://www.utcourts.gov/mediation/cpm/docs/CPM-Motion_To_Dismiss.pdf
Downloading: 8791454522779 https://www.utcourts.gov/howto/family/gc/conservatorship/minor/10_Acceptance.pdf
Downloading: 8791454509243 https://www.utcourts.gov/howto/family/gc/conservatorship/minor/02_Accounting_Minor.pdf
Downloading: 8791404019011 https://www.utcourts.gov/howto/family/gc/docs/1251GE_Certification_of_Readiness_for_Trial_Probate.pdf
Downloading: 8791454474435 https://www.utcourts.gov/howto/family/gc/conservatorship/minor/07_Findings_Conclusions.pdf
Downloading: 8791437746523 https://www.utcourts.gov/howto/family/gc/conservatorship/minor/01_Inventory_Minor.pdf
Downloading: 8791454473499 https://www.utcourts.gov/howto/family/gc/conservatorship/minor/11_Letter.pdf
Downloading: 8791454474435 https://www.utcourts

Downloading: 8791454509139 https://www.utcourts.gov/howto/landlord/docs/1040EV_3_Day_Notice_to_Vacate_for_Committing_Criminal_Act_on_Premises.pdf
Downloading: 8791454393651 https://www.utcourts.gov/howto/landlord/docs/1025EV_3_Day_Notice_to_Vacate_for_Committing_Waste.pdf
Downloading: 8791454474387 https://www.utcourts.gov/howto/landlord/docs/1010EV_3_Day_Notice_to_Vacate_for_Criminal_Nuisance.pdf
Downloading: 8791454522795 https://www.utcourts.gov/howto/landlord/docs/1030EV_3_Day_Notice_to_Vacate_for_Unlawful_Business.pdf
Downloading: 8791437746515 https://www.utcourts.gov/howto/landlord/docs/1035EV_3_Day_Notice_to_Vacate_for_Lease_Violation_Which_Cannot_be_Brought_into_Compliance.pdf
Downloading: 8791454367139 https://www.utcourts.gov/howto/landlord/docs/1015EV_3_Day_Notice_to_Vacate_for_Nuisance.pdf
Downloading: 8791454555419 https://www.utcourts.gov/howto/expunge/docs/09_Acceptance_of_service.pdf
Downloading: 8791437746515 https://www.utcourts.gov/howto/expunge/docs/09_Consent-waiv

Downloading: 8791454473715 https://www.utcourts.gov/resources/forms/garnishment/docs/1504DC_Writ_of_Garnishment_and_Instructions.pdf
Downloading: 8791454367147 https://www.utcourts.gov/resources/interp/docs/Formal_Complaint_Form.pdf
Downloading: 8791404019059 https://www.utcourts.gov/resources/interp/docs/Formal_Complaint_Form-Spanish.pdf
Downloading: 8791454367139 https://www.utcourts.gov/howto/judgment/docs/1003DC_Abstract_of_Judgment.pdf
Downloading: 8791454473507 https://www.utcourts.gov/howto/filing/motions/docs/1104GE_Memorandum_Opposing_Motion.pdf
Downloading: 8791454367139 https://www.utcourts.gov/howto/family/Temporary_Order/docs/1101FA_Motion_for_Temporary_Order_no_Children.pdf
Downloading: 8791454473475 https://www.utcourts.gov/howto/family/Temporary_Order/docs/1102FA_Motion_for_Temporary_Order_with_Children.pdf
Downloading: 8791454473475 https://www.utcourts.gov/howto/family/Temporary_Order/docs/1103FA_Order_on_Motion_for_Temporary_Order_no_Children.pdf
Downloading: 8791454

Downloading: 8791454555507 https://www.utcourts.gov/howto/filing/non-public/docs/1052GE_Nonpublic_Information_Minors.pdf
Downloading: 8791454509139 https://www.utcourts.gov/howto/filing/non-public/docs/1051GE_Nonpublic_Information_Parent.pdf
Downloading: 8791454474435 https://www.utcourts.gov/howto/filing/non-public/docs/1053GE_Nonpublic_Information_Safeguard.pdf
Downloading: 8791454555603 https://www.utcourts.gov/howto/family/modification/docs/1134FA_Notice_of_Modification.pdf
Downloading: 8791454555603 https://www.utcourts.gov/howto/family/modification/child_support/docs/motion/1139FA_Order_on_Motion_to_Adjust_Child_Support.pdf
Downloading: 8791454509131 https://www.utcourts.gov/howto/family/modification/child_support/docs/petition/1137FA_Order_on_Petition_to_Modify_Child_Support.pdf
Downloading: 8791454555547 https://www.utcourts.gov/howto/family/modification/child_support/docs/petition/1135FA_Petition_and_Stipulation_to_Modify_Child_Support.pdf
Downloading: 8791454555547 https://ww

Downloading: 8791454473531 https://www.utcourts.gov/howto/criminallaw/docs/1502CR_Order_on_Motion_to_Remove_Link_to_Personal.pdf
Downloading: 8791403943795 https://www.utcourts.gov/howto/judgment/renew/docs/1103DC_Findings_Conclusions_Order_Motion_to_Renew_Judgment.pdf
Downloading: 8791454473531 https://www.utcourts.gov/howto/judgment/renew/docs/1102DC_Memorandum_Opposing_Motion_to_Renew_Judgment.pdf
Downloading: 8791403943771 https://www.utcourts.gov/howto/judgment/renew/docs/1101DC_Motion_and_Affidavit_to_Renew_Judgment.pdf
Downloading: 8791454555467 https://www.utcourts.gov/howto/judgment/set_aside/docs/1070GE_Motion_to_Set_Aside_Default_or_Judgment.pdf
Downloading: 8791454509131 https://www.utcourts.gov/howto/judgment/set_aside/docs/1071GE_Order_on_Motion_to_Set_Aside_Default_or_Judgment.pdf
Downloading: 8791403943683 https://www.utcourts.gov/howto/filing/motions/dl_suspension/docs/05_Abstract_of_Records.pdf
Downloading: 8791454509131 https://www.utcourts.gov/howto/filing/motions/d

Downloading: 8791403943691 https://www.utcourts.gov/howto/civil/intervention/docs/1004IC_Order_on_Petition_for_Essential_Treatment.pdf
Downloading: 8791403943907 https://www.utcourts.gov/howto/civil/intervention/docs/1002IC_Order_on_Request_for_Examination.pdf
Downloading: 8791454473531 https://www.utcourts.gov/howto/civil/intervention/docs/1001IC_Petition_for_Essential_Treatment.pdf
Downloading: 8791403943691 https://www.utcourts.gov/howto/civil/intervention/docs/1003IC_Request_for_Hearing_Petition_for_Essential_Treatment.pdf
Downloading: 8791403943699 https://vitalrecords.health.utah.gov/wp-content/uploads/Order-Establishing-Fact-of-Birth-adult.pdf
Downloading: 8791454555611 https://www.utcourts.gov/howto/family/fact_of_birth/docs/01_Petition_for_Order_Establishing_Fact_of_Birth_Adult.pdf
Downloading: 8791454473563 https://www.utcourts.gov/howto/family/fact_of_birth/docs/01_Petition_for_Order_Establishing_Fact_of_Birth_Minor.pdf
Downloading: 8791454555611 https://www.utcourts.gov/how

Downloading: 8791403943723 https://www.utcourts.gov/abuse/docs/08_Request_To_Modify_Protective_Order.pdf
Downloading: 8791454555531 https://www.utcourts.gov/abuse/docs/1206PO_Request_to_Modify_Sexual_Violence_Protective_Order.pdf
Downloading: 8791437746979 https://www.utcourts.gov/abuse/docs/13_Respondents_Request_to_Dismiss_Protective_Order.pdf
Downloading: 8791454523403 https://www.utcourts.gov/abuse/docs/12_Respondents_Request_to_Vacate_Temporary_Protective_Order.pdf
Downloading: 8791403943739 https://www.utcourts.gov/abuse/docs/18_1_Return_of_Service.pdf
Downloading: 8791403943739 https://www.utcourts.gov/abuse/docs/1203PO_Sexual_Violence_Protective_Order.pdf
Downloading: 8791454523403 https://www.utcourts.gov/abuse/docs/Temporary_Dating_Violence_Protective_Order.pdf
Downloading: 8791403943731 https://www.utcourts.gov/abuse/docs/03_Temporary_Protective_Order.pdf
Downloading: 8791454523403 https://www.utcourts.gov/abuse/docs/1202PO_Temporary_Sexual_Violence_Protective_Order.pdf
Down

Downloading: 8791454388659 https://www.utcourts.gov/resources/forms/subpoena/docs/1221GE_Notice_to_Persons_Served_with_a_Subpoena.pdf
Downloading: 8791454555531 https://www.utcourts.gov/resources/forms/subpoena/docs/1222GE_Objection_to_Subpoena.pdf
Downloading: 8791403943731 https://www.utcourts.gov/resources/forms/subpoena/docs/1220GE_Subpoena.pdf
Downloading: 8791437747107 https://www.utcourts.gov/howto/filing/summons/docs/1105EV_Summons_Eviction.pdf
Downloading: 8791437747003 https://www.utcourts.gov/howto/filing/summons/docs/1105EV_Summons_Eviction_Chinese.pdf
Downloading: 8791454555531 https://www.utcourts.gov/howto/filing/summons/docs/1105EV_Summons_Eviction_Vietnamese.pdf
Downloading: 8791454555531 https://www.utcourts.gov/howto/filing/summons/docs/1015GE_Summons_In_State_Arabic.pdf
Downloading: 8791454370139 https://www.utcourts.gov/howto/filing/summons/docs/1015GE_Summons_In_State_Chinese.pdf
Downloading: 8791454370235 https://www.utcourts.gov/howto/filing/summons/docs/1015GE_

KeyboardInterrupt: 

In [42]:
display(files_df)

,source,title,group,url,filename,downloaded
0,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-03
1,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-03
2,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-03
3,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-03
4,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-03
...,...,...,...,...,...,...
592,www.utcourts.gov,Fifth District Request for Copy of Video/Audio...,The Fifth District Court,https://www.utcourts.gov/courts/dist/distsites...,video_audio_request.pdf,2021-11-03
593,www.utcourts.gov,First District Request for Audio Recording,The First Judicial District,https://www.utcourts.gov/courts/dist/distsites...,audio_request_form.pdf,2021-11-03
594,www.utcourts.gov,Domestic Status Conference Worksheet,The Fourth Judicial District,https://www.utcourts.gov/courts/dist/distsites...,Domestic%20Status%20Conference%20Worksheet.pdf,2021-11-03
595,www.utcourts.gov,Fourth District Bail Refund Request,The Fourth Judicial District,https://www.utcourts.gov/courts/dist/distsites...,Bail%20Refund%20Request%20Form%204th%20Distric...,2021-11-03


In [40]:
files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")

# Download forms and meta data from Michigan courts

Download forms and place them in the folder defined by `form_path` while creating a dataframe that you save to `csv_path`.


In [12]:
#from requests_html import HTMLSession
#from requests_html import AsyncHTMLSession
#import nest_asyncio
#import pyppdf.patch_pyppeteer

#asession = AsyncHTMLSession()
#r = await asession.get("https://www.courts.michigan.gov/SCAO-forms/appeals/")
#await r.html.arender(timeout=15, sleep=5)

In [64]:
source = "www.courts.michigan.gov"
form_path = "../data/raw/www.courts.michigan.gov/forms/"
csv_path = "../data/raw/www.courts.michigan.gov/"

today = date.today().strftime("%Y-%m-%d")

try:
    files_df = pd.read_csv(csv_path+"form_data.csv")
except:
    files_df = pd.DataFrame([],columns=["source","title","group","url","filename","downloaded"])
    files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")

The MI court site is all js rendered. So, I downloaded the rendered pages into `../data/raw/www.courts.michigan.gov/SCAO-forms/html/` and will deal with them there. 

In [95]:
def MI_get(file):

    group = reCase(re.search(".*/([^/]*)\.html",file, re.IGNORECASE).group(1))

    with open(file) as f:
        contents = f.read()
    f.close()
    soup = BeautifulSoup(contents)
    body = soup.find('div',id="main-content")

    for a in body.find_all('a', href=True):
        if (re.search(".*\.pdf",a['href'])):
            title = a.text.strip()
            url = re.search(".*/([^/]*)\.pdf",a['href'], re.IGNORECASE)
            filename = url.group(1)+".pdf"
            fileurl = "https://www.courts.michigan.gov"+url.group(0)

            #print("\t",title,group,filename,"\n\t",fileurl,"\n")

            status = ""
            if fileurl not in files_df["url"].values:
                print(i,"Downloading: "+fileurl)
                #status = download_pdf(fileurl,form_path,str(hash(url))+".pdf")

                if status == "error":
                    print("#######################")
                    print("         Error!!!")
                    print("#######################")
                    time.sleep(5)
                else:
                    i += 1
                    #files_df = files_df.append(pd.DataFrame([[source,title,group,fileurl,filename,today]],columns=["sourcesoruce","title","group","url","filename","downloaded"]), ignore_index=True,sort=False)
                    #time.sleep(1)

In [66]:
MI_get("../data/raw/www.courts.michigan.gov/SCAO-forms/html/LTLC-forms.html")

In [18]:
path = "../data/raw/www.courts.michigan.gov/html/"
f = []
for (dirpath, dirnames, filenames) in walk(path):
    f.extend(filenames)
    break
f1 = [file for file in f]

for file in f1:
    if re.match("\.pdf$",file):
        print(file)

In [ ]:
display(files_df)

In [ ]:
files_df.to_csv(csv_path+"_form_data.csv", index=False, encoding="utf-8")